In [1]:
import keras
from keras import layers
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import codecs
import random
import sys

In [2]:
with codecs.open('avatar_fulltextjson.json','r','utf-8-sig') as data:
    text = json.load(data).lower()

In [3]:
print('Corpus Length:',len(text))

Corpus Length: 1814811


In [4]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 604917
Unique characters: 55
Vectorization...


<ipython-input-4-414f9227a6c5>:26: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
<ipython-input-4-414f9227a6c5>:27: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


In [5]:
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [6]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

C:\Users\chels\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [7]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [8]:
for epoch in range(1, 10):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.5, 0.75, 1.0]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 1000 characters
        for i in range(1000):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
4726/4726 [==============================] - 306s 64ms/step - loss: 1.6034
--- Generating with seed: "ocean surface. a fish appears out of nowhere and slaps sokka"
------ temperature: 0.5
ocean surface. a fish appears out of nowhere and slaps sokka. cuts to shot of the panners. cut to a shot of the group and starts from the statue of the shot of aang with be sokka and you the coming the camera pans to the through the fire blasts. sokka and katara walks to the toph. aang stares and is starts the carrials. [and while the fire lord's foot to aang and passes a contreding the prince of a smile the water to see the statue. i'm make it falls of the stands of stor before katara and the tinning are and can't starts around the way. [sits the closer of the swide shows the rests look on the scene off and the camera cuts to a shot of the coress and katara lise-up and aang with the ship. a fire blast of the streat before the stands of the than stands on the tire of the topher in sokka and th

<ipython-input-7-66c6eecb696d>:3: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


ollow and the avatar. you're the way to ad,it counter the fire falls of trailed. she is using inside the river. cut to a close-up with an katara is sitting at zoom to katara and sokka and his basther shappiors from a rock and is atto appear switch to aang. sokka snows it around a broth wardened just as more amount fellows a damined and filled back from the process. [surprised the glow which aang and sokka and sokka standing of the care to the approaching over a facing with a large stice in front of her. cut to a side-view of the blade from the place and places the camera closer he looks at the background.] a way a bast warmen lord wouldn't have to remend teach nice her all it so not sthe knee meats time to stare in crawl over. of warms up the avat
------ temperature: 1.0
knee meats time to stare in crawl over. of warms up the avatar akoda, where he aseress-watch, mountains beclefuls, only charge. air tenig imagate. [cut to shot.] aang disuges one raise! [comes speaking on the wardended